Managing Seeds
==============

This quick tutorial explains how to deal with input files manipulated and generated during the execution.
Let's reuse the following base snippet. The only difference here is that we are going to change the coverage
strategy to `PATH_COVERAGE`.

In [1]:
from tritondse import SymbolicExecutor, Config, Seed, Program, ProcessState, SymbolicExplorator, CoverageStrategy

config = Config(symbolize_argv=True, pipe_stdout=False, coverage_strategy=CoverageStrategy.PATH_COVERAGE)

def post_exec_hook(se: SymbolicExecutor, state: ProcessState):
    print(f"seed:{se.seed.hash} ({repr(se.seed.content)})   [exitcode:{se.exitcode}]")

dse = SymbolicExplorator(config, Program("crackme_xor"))

dse.add_input_seed(Seed(b"./crackme AAAAAAAAAAAAAAA"))

dse.callback_manager.register_post_execution_callback(post_exec_hook)

dse.explore()

seed:78fd4aa0744187fcda352908d6263e3b (b'./crackme AAAAAAAAAAAAAAA')   [exitcode:0]
seed:ba39b0af614b34616b62e732d2cd2c3f (b'./crackme eAAAAAAAAAAAAAA')   [exitcode:0]
seed:9ea41b172ebd4b1a3b0840559c79936e (b'./crackme elAAAAAAAAAAAAA')   [exitcode:0]
seed:1a997756f724d8ba6700c88049d082e7 (b'./crackme eliAAAAAAAAAAAA')   [exitcode:0]
seed:9b2a08c3a7d7dc2659a0de22572a046d (b'./crackme elitAAAAAAAAAAA')   [exitcode:0]
seed:bee1ba2ede4da2a0df49cf8e682cd163 (b'./crackme eliteAAAAAAAAAA')   [exitcode:0]


<ExplorationStatus.IDLE: 2>

Just by exploring all possible paths we managed to solve the challenge. Let's play
with the corpus.

## Initial Corpus

There are two ways to provide an initial corpus:

* providing in existing workspace directory and putting manually files in the *worklist* directory
* via the API by adding the seed with `add_input_seed`, it will automatically be added in seeds to process.

## Managing generated corpus

**SymbolicExecutor**: This class is solely meant to execute a single seed, not to produce
new ones *(that is the purpose of the explorator)*. However if a one wants to generate a
new input to process in a callback, it is possible with the method `enqueue_seed`. That
method will just fill a list that will later be picked-up by the `SymbolicExplorator` instance.


Here is a dummy hook function that manually negate a branching condition to generate a new input file:

In [17]:
def hook_cmp(se: SymbolicExecutor, pstate: ProcessState, addr: int):
    zf = pstate.cpu.zf  # concrete value
    sym_zf = pstate.read_symbolic_register(pstate.registers.zf)
    
    # Revert the current value of zf to "negate" condition
    status, model = pstate.solve(sym_zf.getAst() != zf)
    
    if status == SolverStatus.SAT:
        new_seed = se.mk_new_seed_from_model(model)
        se.enqueue_seed(new_seed)

By default the status of the seed generated is `FRESH` but the seed you generated is a faulty one, you can directly assign it the `CRASH` status.

**SymbolicExplorator**: All seeds generated during the exploration are moved in the `Workspace` instance from the *worklist* folder to the appropriate one *corpus*, *hangs* and *crashs*.
One can iterate them using the workspace instance.

In [9]:
print("seed to process:", len(list(dse.workspace.iter_worklist())))

print("\nCorpus:")
for seed in dse.workspace.iter_corpus():
      print(seed.filename)

seed to process: 0

Corpus:
1a997756f724d8ba6700c88049d082e7.00000019.tritondse.cov
9b2a08c3a7d7dc2659a0de22572a046d.00000019.tritondse.cov
9ea41b172ebd4b1a3b0840559c79936e.00000019.tritondse.cov
bee1ba2ede4da2a0df49cf8e682cd163.00000019.tritondse.cov
ba39b0af614b34616b62e732d2cd2c3f.00000019.tritondse.cov
78fd4aa0744187fcda352908d6263e3b.00000019.tritondse.cov


## Setting seed status

During the execution one, can assign a status to the seed currently being executed in any callback.
*(Make sure not to override a status previously set by another callback)*. At the end of an execution
of no status has been assigned, the seed is automatically assigned `OK_DONE`.